## Przygotowanie środowiska i danych

In [23]:
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
from sklearn import metrics
import matplotlib.pyplot as plt
plt.style.use('seaborn-ticks')
%matplotlib inline

In [24]:
df_o = pd.read_csv("data/bank-balanced.csv")
print(df_o.shape)
df_o.head()

(11162, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [25]:
df_o.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'deposit'],
      dtype='object')

In [26]:
target = 'deposit'
numFeatures = ['age', 'balance', 'duration', 'campaign', 'pdays','previous', 'day']
catFeatures = ['job', 'marital', 'education', 'default',  'housing', 'loan', 'contact',  'month', 'poutcome']

In [27]:
df_o[feature]

0        unknown
1        unknown
2        unknown
3        unknown
4        unknown
          ...   
11157    unknown
11158    unknown
11159    unknown
11160    failure
11161    unknown
Name: poutcome, Length: 11162, dtype: object

In [28]:
from sklearn import preprocessing

In [29]:
# Kopiowanie obiektu
df = df_o.copy()
# Słownik zawieracy mapy
mapy = {}
for feature in catFeatures:
# Iniciujemy obiekt do kodowania, który będzie przechowywał mapę
    le = preprocessing.LabelEncoder()
# Dopasowujemy kolumnę (tworzymy mapę) i od razu ją nakładamy na zmienną, na której robiliśmy dopasowanie
    df[feature] = le.fit_transform(df_o[feature])
# Zapisujemy mapę, aby móc odzyskać informację o mapowaniu i oryginalnych wartościach   
    mapy[feature] = le

In [20]:
print(features)
features = df.columns.tolist()
print(features)
features.remove(target)
print(features)

['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit']
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']


In [32]:
# Jakie klasy się utworzyły dla zmiennej  married
print(mapy['marital'].classes_)
# Odwrócenie mapowania
print(mapy['marital'].inverse_transform(df['marital']))
print(df_o["marital"].values)

['divorced' 'married' 'single']
['married' 'married' 'married' ... 'single' 'married' 'married']
['married' 'married' 'married' ... 'single' 'married' 'married']


## Przygotowanie wrappera do walidacji krzyżowej
Bazujemy na procedurze wykorzystanej do walidacji kNN. 
Stosujemy:
```clf = RandomForestClassifier(*args, **kwargs)``` 

In [38]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score


# Przygotujmy wrapper
def CVTestRFClass(nFolds = 5, randomState=2020, debug=False, features=features, *args, **kwargs):
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)

    # listy do przechowywania wyników:
    testResults = []
    trainResults = []
    predictions = []
    indices = []

    # Pętla walidująca model na kolejnych foldach
    for train, test in kf.split(df.index.values):
        # Przygotowanie estymatora
        clf = RandomForestClassifier(*args, **kwargs, random_state=randomState, n_jobs=-1)
        if debug:
            print(clf)
        # Trenowanie modelu
        clf.fit(df.iloc[train][features], df.iloc[train][target])

        # Przygotowanie prognoz dla zbioru treningowego i testowego
        # Sklearn zwraca dwie kolumny prawdopodobieństw dla obydwu klas
        predsTrain = clf.predict_proba(df.iloc[train][features])[:,1]
        preds = clf.predict_proba(df.iloc[test][features])[:,1]
        
        # Informacje o predykcjach dla tego foldu
        predictions.append(preds.tolist().copy())
        
        # Indeksy w oryginalnym data frame
        indices.append(df.iloc[test].index.tolist().copy())
        
        # Policzenie dopasowania za pomocą metryki ROC-AUC
        trainScore = roc_auc_score((df[target].iloc[train]=="yes").astype(int), predsTrain)
        testScore = roc_auc_score((df[target].iloc[test]=="yes").astype(int), preds)
        
        # Zapisanie wyników do listy 
        trainResults.append(trainScore)
        testResults.append(testScore)
        
        # Informowanie o każdym foldzie razem z wynikami treningowymi możemy opcjonalnie wyświetlać w trakcie
        if debug:
            print("Train AUC:", trainScore,
                  "Valid AUC:", testScore)
        
    return trainResults, testResults, predictions, indices


## Pierwsze uruchomienie i przegląd hiperparametrów

In [39]:
trainResults, testResults, predictions, indices = CVTestRFClass(debug=True)
print(np.mean(testResults))

RandomForestClassifier(n_jobs=-1, random_state=2020)
Train AUC: 1.0 Valid AUC: 0.9120633918342885
RandomForestClassifier(n_jobs=-1, random_state=2020)
Train AUC: 1.0 Valid AUC: 0.9206705884815203
RandomForestClassifier(n_jobs=-1, random_state=2020)
Train AUC: 1.0 Valid AUC: 0.9211460733577608
RandomForestClassifier(n_jobs=-1, random_state=2020)
Train AUC: 1.0 Valid AUC: 0.9108430127451866
RandomForestClassifier(n_jobs=-1, random_state=2020)
Train AUC: 1.0 Valid AUC: 0.9215899153297145
0.9172625963496941


Uzyskanie w każdym przypadku dopasowania 100% na zbiorze treningowym. Drzewa w lesie losowym bardzo mocno się przetrenowywały (dowolna głębokość). Zwiększamy liczbe drzew.

In [40]:
# Pętla po parametrze n_estimators
for k in [5, 10, 25, 50, 100, 200, 500, 1000]:
    trainResults, testResults, predictions, indices = CVTestRFClass(n_estimators=k)
    print(k, np.mean(trainResults), np.mean(testResults), np.mean(trainResults) - np.mean(testResults))

5 0.9967996180817484 0.8781426914084364 0.11865692667331207
10 0.9995838573442228 0.8977571818373228 0.10182667550690006
25 0.9999872889879888 0.9104520709909532 0.08953521799703568
50 0.9999998893082228 0.915498712452151 0.08450117685607184
100 1.0 0.9172625963496941 0.08273740365030591
200 1.0 0.91835310780829 0.08164689219170995
500 1.0 0.9185454575127533 0.0814545424872467
1000 1.0 0.9186900993261761 0.08130990067382393


Jak widać Random Forest nie przetrenowuje się mocniej wraz ze wzrostem liczby drzew. 
Są od siebie w pełni niezależne. 
Jednocześnie obserwujemy malejące krańcowe korzyści płynące z większej liczby drzew. 
Pozostajemy na 100 drzewach na czas dalszych eksperymentów. 

In [41]:
for k in range(2,22,2):
    trainResults, testResults, predictions, indices = CVTestRFClass(n_estimators=100, max_depth=k)
    print(k, np.mean(trainResults), np.mean(testResults), np.mean(trainResults) - np.mean(testResults))

2 0.8607333808146999 0.8581649560975702 0.0025684247171297026
4 0.8936940355103637 0.8872820188275904 0.006412016682773358
6 0.9169055075144918 0.9018813751485426 0.015024132365949239
8 0.9425071143421799 0.909976013149403 0.03253110119277691
10 0.9674590740898426 0.9149029967758908 0.05255607731395173
12 0.9859798340300066 0.9159709344584902 0.07000889957151635
14 0.9960079791951113 0.9174383296955 0.07856964949961132
16 0.9994779649755341 0.9174648432442097 0.08201312173132436
18 0.9999780952041168 0.9174353744197397 0.08254272078437719
20 0.9999998994206433 0.9170143607627443 0.08298553865789893
